## Model 1 - ERC (Emotion Recognition in conversation)

#### Setup

In [ ]:
# ! pip install transformers
# ! pip install sentence_transformers

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

import pickle
import os

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data *engneering*

In [ ]:
# folder_path = "/content/drive/MyDrive/IIITD/Courses/nlp/Assignment 4/"
# path_train_file = folder_path+"train_file.json"
# df_train = pd.read_json(path_train_file)
# df_train.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you you had sex with Ursula?!, Uh, a litt...","[1, 1, 0, 0, 0]"
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0, 0, 0, 0, 1, 1, 1]"
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0, 0, 1]"
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0, 0, 1]"


In [ ]:
# for i in df_train['utterances']:
#     print(i)
#     break ;

['You-you\x85you had sex with Ursula?!', 'Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and', "You didn't notice she was wearing different clothes?!", 'Well I was just so excited to see you.', "Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."]


In [ ]:
# model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
# df_train['utterances_embeddings'] = np.nan

# df_train.head()

In [ ]:
# if os.path.exists('sentence_embeddings.pkl'):
#     embeddings = pickle.load(open('sentence_embeddings.pkl', 'rb'))
# else :
#     embeddings = [model.encode(utterance) for utterance in tqdm(df_train['utterances'])]
#     pickle.dump(embeddings, open('sentence_embeddings.pkl', 'wb'))

# df_train['utterances_embeddings'] = embeddings

# df_train.head()

In [ ]:
# df_train.info()

In [ ]:
# def generate_cnn_embeddings(row):
#     speakers = row['speakers']

#     embeddings_list = row['utterances_embeddings']

#     speaker_index = {}
#     index = 0
#     for speaker in speakers:
#         if speaker not in speaker_index:
#             speaker_index[speaker] = index
#             index += 1

#     S = len(speaker_index)
#     N = len(speakers)
#     embeddings_tensor = np.zeros((max(10 , S), 768, N))
#     for i, (speaker, embedding) in enumerate(zip(speakers, embeddings_list)):
#         idx = speaker_index[speaker]
#         embeddings_tensor[idx, :, i] = embedding
#     return embeddings_tensor


# df_train['CNN_embeddings'] = df_train.apply(generate_cnn_embeddings, axis=1)

In [ ]:
# pickle.dump(df_train, open('df_train_with_CNN_embeddings.pkl', 'wb'))

In [ ]:
# df_train = pickle.load(open('df_train_with_CNN_embeddings.pkl', 'rb'))

# df_train.head()

,episode,speakers,emotions,utterances,triggers,utterances_embeddings,CNN_embeddings
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you you had sex with Ursula?!, Uh, a litt...","[1, 1, 0, 0, 0]","[[0.020053696, 0.048883013, -0.037799813, -0.0...","[[[0.020053695887327194, 0.0, -0.0219856929033..."
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[0.0045765517, 0.025685206, -0.02977001, 0.02...","[[[0.004576551727950573, 0.0251162052154541, 0..."
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0, 0, 0, 0, 1, 1, 1]","[[0.063676834, 0.084198244, 0.0095247915, -0.0...","[[[0.06367683410644531, 0.0, 0.0, 0.0, 0.0, 0...."
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0, 0, 1]","[[-0.00056250574, 0.07043089, -0.0339791, -0.0...","[[[-0.0005625057383440435, 0.02774389833211898..."
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0, 0, 1]","[[0.043867074, 0.019203804, -0.011715968, -0.0...","[[[0.0438670739531517, 0.0, 0.0092357126995921..."


### **Models**

### DataLoader

In [27]:
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self, json_path = 'train_file.json', spacial_speaker_embedding=False):
        self.embedding_model = SentenceTransformer('all-mpnet-base-v2')
        self.spacial_speaker_embedding=spacial_speaker_embedding
        self.train_data = pd.read_json(json_path)
        emotions = ['surprise', 'neutral', 'disgust', 'fear', 'sadness', 'anger', 'joy']
        self.emotion_dict = {}
        for i, emotion in enumerate(emotions):
            self.emotion_dict[emotion] = i

    def generate_cnn_embeddings(self, row):
        speakers = row['speakers']

        embeddings_list = self.embedding_model.encode(row['utterances'])

        speaker_index = {}
        index = 0
        for speaker in speakers:
            if speaker not in speaker_index:
                speaker_index[speaker] = index
                index += 1

        S = len(speaker_index)
        N = len(speakers)
        embeddings_tensor = np.zeros((max(10 , S), N, 768))
        for i, (speaker, embedding) in enumerate(zip(speakers, embeddings_list)):
            idx = speaker_index[speaker]
            embeddings_tensor[idx, i, :] = embedding
        return embeddings_tensor

    def generate_embeddings(self,row):
        embeddings_list = self.embedding_model.encode(row['utterances'])
        return embeddings_list

    def gen_emotion_matrix(self, row):
        emotions = row['emotions']
        emotion_matrix = np.zeros((len(emotions), 7))
        for idx, emotion in enumerate(emotions):
            emotion_matrix[idx][self.emotion_dict[emotion]] = 1
        return emotion_matrix


    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        if self.spacial_speaker_embedding:
            X = self.generate_cnn_embeddings(self.train_data.iloc[idx])
        else :
            X = self.generate_embeddings(self.train_data.iloc[idx])
        Y = self.gen_emotion_matrix(self.train_data.iloc[idx])
        return X, Y

### Model

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ERC_Attention(nn.Module):
    def __init__(self, num_heads=8, input_size=768, hidden_size=64, num_layers=1, mlp_hidden_size=64, output_size=7):
        super(ERC_Attention, self).__init__()
        """
        EERC_Attention Module combining Attention, BiLSTM, and MLP
        Input shape: (batch_size, in_channels, width, length)
        Output shape: (batch_size, seq_length, output_size)
        """
        # self.input_size = input_size
        self.hidden_size = hidden_size
        # Attention layers
        self.num_heads = num_heads
        self.multihead_attn = nn.MultiheadAttention(input_size, num_heads=num_heads)

        # BiLSTM layer
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # MLP layers
        self.fc1 = nn.Linear(hidden_size * 2, mlp_hidden_size)
        self.fc2 = nn.Linear(mlp_hidden_size, output_size)
        self.softmax = nn.Softmax(dim = 1)


    def forward(self, x):
        # Attention
        utterance_embeddings = x
        x, _ = self.multihead_attn(utterance_embeddings, utterance_embeddings, utterance_embeddings)

        # BiLSTM
        x = x.squeeze(1)  # Remove the singleton dimension
        x, _ = self.bilstm(x)  # out shape: (batch_size, seq_length, hidden_size * 2)

        # MLP
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        # x = self.softmax(x) # softmax is not letting it train fast atleast

        return x

class ERC_CNN(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, input_size, hidden_size, num_layers, mlp_hidden_size, output_size):
        super(ERC_CNN, self).__init__()
        """
        ERC-CNN Module combining CNN, BiLSTM, and MLP
        Input shape: (batch_size, in_channels, width, length)
        Output shape: (batch_size, seq_length, output_size)
        """
        # self.input_size = input_size
        self.hidden_size = hidden_size
        # self.num_layers = num_layers
        # CNN layers
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=(3, 3), padding=1)

        # BiLSTM layer
        # self.h0, self.c0 = torch.zeros(self.num_layers * 2, self.input_size, self.hidden_size), torch.zeros(self.num_layers * 2, self.input_size, self.hidden_size)
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        # MLP layers
        self.fc1 = nn.Linear(hidden_size * 2, mlp_hidden_size)
        self.fc2 = nn.Linear(mlp_hidden_size, output_size)
        self.softmax = nn.Softmax(dim = 1)


    def forward(self, x):
        # CNN
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)

        # BiLSTM
        x = x.squeeze(1)  # Remove the singleton dimension
        x, _ = self.bilstm(x)  # out shape: (batch_size, seq_length, hidden_size * 2)

        # MLP
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        # x = self.softmax(x) # softmax is not letting it train fast atleast

        return x


In [66]:
# Define parameters
batch_size = 1
speakers = 10  # max_speakers
num_utterances = 24  # dialogue_length
embedding_size = 768

cnn_mid_channels = 3
cnn_out_channels = 1

hidden_lstm = 64
layers_lstm = 4

inputs_mlp = hidden_lstm * 2
hidden_mlp = 64
output_mlp = number_of_emotions = 7


# Create individual components
# Initialize the ERC_CNN class
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ERC_CNN(in_channels=speakers,
                mid_channels=cnn_mid_channels,
                out_channels=cnn_out_channels,
                input_size=embedding_size,
                hidden_size=hidden_lstm,
                num_layers=layers_lstm,
                mlp_hidden_size=hidden_mlp,
                output_size=output_mlp).to(DEVICE)

num_heads = 2
model_2 = ERC_Attention(num_heads=num_heads,
                input_size=embedding_size,
                hidden_size=hidden_lstm,
                num_layers=layers_lstm,
                mlp_hidden_size=hidden_mlp,
                output_size=output_mlp).to(DEVICE)

# input_tensor = torch.randn(batch_size, num_utterances, embedding_size)
# # output = model(input_tensor)
# output = model_2(input_tensor)
# print("Final Output shape:", output.shape)

In [ ]:
# Define your model, loss function, and optimizer
folder_path = "/content/drive/MyDrive/IIITD/Courses/nlp/Assignment 4/"
# folder_path = ""
path_train_file = folder_path+"train_file.json"
path_val_file = folder_path+"val_file.json"

spacial_speaker_embedding = True
train_dataset = MyDataset(path_train_file,spacial_speaker_embedding)
val_dataset = MyDataset(path_val_file,spacial_speaker_embedding)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(DEVICE)
torch.save(model.state_dict(), 'cnn.pth')
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)

from tqdm import tqdm
num_epochs = 10
for epoch in range(num_epochs):
    model.load_state_dict(torch.load('cnn.pth'))
    running_loss = 0.0
    model.train()
    for i, (inputs, labels) in tqdm(enumerate(train_dataloader)):
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = model(inputs.float())
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    # Print average loss for the epoch
    epoch_loss = running_loss / len(train_dataloader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    torch.save(model.state_dict(), 'cnn.pth')

    if(epoch%3==2):
        model.eval()
        val_running_loss = 0.0
        for i, (inputs, labels) in tqdm(enumerate(val_dataloader)):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            # optimizer.zero_grad()  # Zero the parameter gradients

            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)

        # Print average loss for the epoch
        val_epoch_loss = val_running_loss / len(val_dataloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_epoch_loss:.4f}")

### Attention

In [ ]:
import torch
import torch.nn as nn

#! add speaker embedding maybe self attention on sparse

class SelfAttention(nn.Module):
    def __init__(self, embedding_size, num_heads):
        super(SelfAttention, self).__init__()
        self.num_heads = num_heads
        self.multihead_attn = nn.MultiheadAttention(embedding_size, num_heads)

    def forward(self, utterance_embeddings):
        # Reshape utterance embeddings to (seq_len, batch_size, embedding_dim)
        # utterance_embeddings = utterance_embeddings.unsqueeze(0)  # Add a dimension for seq_len
        output, _ = self.multihead_attn(utterance_embeddings, utterance_embeddings, utterance_embeddings)
        return output

# Example usage
# num_utterances = 24  # Number of utterances
# # Generate random utterance embeddings
# utterance_embeddings = torch.randn(1, num_utterances, embedding_size)

s = 10  # max_speakers
n = 24  # dialogue_length
num_heads = 8  # Number of attention heads
embedding_size = 768  # Size of each embedding
input = torch.randn(s, n, embedding_size)


# Define self-attention layer
attention_layer = SelfAttention(embedding_size=embedding_size, num_heads=num_heads)

# Apply self-attention
context_embeddings = attention_layer(input)

# Check the shape of the output tensor
print(context_embeddings.shape)  # Output shape: (n, embedding_dim)

torch.Size([10, 24, 768])


### Bi-LSTM

In [ ]:
import torch
import torch.nn as nn

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Bidirectional LSTM layer
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # 2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM : (h_n, c_n): A tuple containing the final hidden state $h_n$ and the final cell state $c_n$ of the LSTM,
        out, _ = self.bilstm(x, (h0, c0))  # out shape: (batch_size, seq_length, hidden_size * 2)

        # Concatenate the outputs from both directions
        out = torch.cat((out[:, :, :self.hidden_size], out[:, :, self.hidden_size:]), dim=2)

        return out

# Define input size, hidden size, and number of layers
input_size = 768
hidden_size = 64
num_layers = 1

n = 24

# Create BiLSTM instance
bilstm = BiLSTM(input_size, hidden_size, num_layers)

# Generate random input tensor
input_tensor = torch.randn(1, n, 768)  # Batch size 24, input size 768
input_tensor = output

# Forward pass through BiLSTM
output = bilstm(input_tensor)  # Add batch dimension

print("Output shape:", output.shape)  # Output shape should be [1, 24, 128] (24 vectors each of size 128)

### MLP + Softmax

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class EmotionMLP(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(EmotionMLP, self).__init__()
#         self.relu = nn.ReLU()
#         self.fc_1 = nn.Linear(input_size, hidden_size)
#         self.fc_2 = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = self.fc_1(x)
#         x = self.relu(x)
#         x = self.fc_2(x)
#         return x


# # Define input size, hidden size, and output size for the MLP
# input_size = 128
# hidden_size = 64
# output_size = 6

# n = 24

# # Create a shared MLP instance
# shared_mlp = EmotionMLP(input_size, hidden_size, output_size)

# # Generate random input tensor
# input_tensor = torch.randn(1, n, 128)  # Number of utterances: n = 24, Utterance embedding size: 128
# input_tensor = output

# # Apply the shared MLP sequentially 24 times
# outputs = []
# for i in range(n):
#     output = shared_mlp(input_tensor[:, i, :])  # Feed each 128-dimensional vector
#     output = F.softmax(output, dim=1)  # Apply softmax along dimension 1
#     outputs.append(output.unsqueeze(1))  # Add a singleton dimension for concatenation later

# # Concatenate the outputs along the second dimension to get 24 6-dimensional outputs
# final_output = torch.cat(outputs, dim=1).squeeze(0)

# print("Final output shape:", final_output)  # Output shape should be [24, 6]

In [ ]:
# import torch
# import torch.nn as nn

# s = 10  # max_speakers
# n = 24  # dialogue_length
# input = torch.randn(s, n, 768)

# # Define the parameters for the convolution operation
# mid_channels = 3
# out_channels = 1  # Number of output channels
# kernel_size = (1, 1, 1)  # Kernel size (depth, height, width)
# stride = 1

# # Define the convolutional layer
# conv3d_1 = nn.Conv3d(in_channels=s, out_channels=mid_channels, kernel_size=kernel_size)
# conv3d_2 = nn.Conv3d(in_channels=mid_channels, out_channels=out_channels, kernel_size=kernel_size)


# # Reshape input to (batch_size, in_channels, depth, height, width)
# input_ = input.unsqueeze(0).unsqueeze(2)  # Adding batch and depth dimensions

# # Perform convolution
# mid = conv3d_1(input_)
# output = conv3d_2(mid)

# print(input.shape)
# print(input_.shape)
# print(mid.shape)
# print(output.shape)

# # Ensure the output has the desired shape
# output = output.squeeze(0).squeeze(1)  # Remove batch and depth dimensions
# print("Output shape:", output.shape)  # Output shape should be 1*n*768

### Attention

In [ ]:
import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, input_size, num_heads):
        super(SelfAttention, self).__init__()
        self.num_heads = num_heads
        self.multihead_attn = nn.MultiheadAttention(input_size, num_heads)

    def forward(self, sentence_embeddings):
        output, _ = self.multihead_attn(sentence_embeddings, sentence_embeddings, sentence_embeddings)
        return output

sentence_embeddings = torch.randn(20, 10, 768)  # Shape: (seq_len, batch_size, embedding_dim)

# Define self-attention layer
num_heads = 1  # Number of attention heads
attention_layer = SelfAttention(input_size=768, num_heads=num_heads)

# Apply self-attention
context_embeddings = attention_layer(sentence_embeddings)

# Check the shape of the output tensor
print(context_embeddings.shape)

In [4]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, embedding_size, num_heads):
        super(SelfAttention, self).__init__()
        self.num_heads = num_heads
        self.multihead_attn = nn.MultiheadAttention(embedding_size, num_heads)

    def forward(self, utterance_embeddings):
        # Reshape utterance embeddings to (seq_len, batch_size, embedding_dim)
        utterance_embeddings = utterance_embeddings.unsqueeze(0)  # Add a dimension for seq_len
        output, _ = self.multihead_attn(utterance_embeddings, utterance_embeddings, utterance_embeddings)
        return output.squeeze(0)  # Remove the added dimension

# Example usage
num_utterances = 10  # Number of utterances
embedding_size = 768  # Size of each embedding
num_heads = 8  # Number of attention heads

# Generate random utterance embeddings
utterance_embeddings = torch.randn(num_utterances, embedding_size)

# Define self-attention layer
attention_layer = SelfAttention(embedding_size=embedding_size, num_heads=num_heads)

# Apply self-attention
context_embeddings = attention_layer(utterance_embeddings)

# Check the shape of the output tensor
print(context_embeddings.shape)  # Output shape: (n, embedding_dim)

torch.Size([10, 768])


In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels):
        super(CNN, self).__init__()
        """
        CNN Module
        Input shape: (batch_size, in_channels, width, length)
        Output shape: (batch_size, out_channels, width, length)
        """
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=(1, 1))
        self.conv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=(1, 1))

    def forward(self, x):
        x = self.conv1(x)
        x = f.relu(x)
        x = self.conv2(x)
        return x

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiLSTM, self).__init__()
        """
        BiLSTM Module
        Input shape: (batch_size, seq_length, input_size)
        Output shape: (batch_size, seq_length, hidden_size * 2)
        """
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        out, _ = self.bilstm(x)
        return out

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        """
        MLP Module
        Input shape: (batch_size, seq_length, input_size)
        Output shape: (batch_size, seq_length, output_size)
        """
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class ERC_CNN(nn.Module):
    # torch.Size([16, 10, 24, 768])
    # torch.Size([16, 1, 24, 768])
    # torch.Size([16, 24, 768])
    # torch.Size([16, 24, 128])
    # torch.Size([16, 24, 128])
    # torch.Size([16, 24, 6])

    # Final Output shape: torch.Size([16, 24, 6])

    def __init__(self, cnn, bilstm, mlp):
        super(ERC_CNN, self).__init__()
        self.cnn = cnn
        self.bilstm = bilstm
        self.mlp = mlp

    def forward(self, x):
        # CNN
        x = self.cnn(x)  # Output shape: (batch_size, out_channels, width, length)

        # BiLSTM
        x = x.squeeze(1)  # Remove the singleton dimension
        x = self.bilstm(x)  # Output shape: (batch_size, seq_length, hidden_size * 2)

        # MLP
        x = self.mlp(x)  # Output shape: (batch_size, seq_length, output_size)

        return x

# Define parameters
batch_size = 1
speakers = 10  # max_speakers
num_utterances = 24  # dialogue_length
embedding_size = 768

cnn_mid_channels = 3
cnn_out_channels = 1

hidden_lstm = 64
layers_lstm = 1

inputs_mlp = hidden_lstm * 2
hidden_mlp = 64
output_mlp = number_of_emotions = 7


# Create individual components
cnn = CNN(speakers, cnn_mid_channels, cnn_out_channels)
bilstm = BiLSTM(embedding_size, hidden_lstm, layers_lstm)
mlp = MLP(inputs_mlp, hidden_mlp, output_mlp)

# # Create the combined model
# model = ERC_CNN(cnn, bilstm, mlp)
# input_tensor = torch.randn(batch_size, speakers, num_utterances, embedding_size)


# # Forward pass through the model
# output = model(input_tensor)  # Add batch dimension

# print("Final Output shape:", output.shape)